Exercise 6: Indexes
===========

Let's play with the [consumer complaint database](https://catalog.data.gov/dataset/consumer-complaint-database) from data.gov

In [1]:
%load_ext sql
%sql sqlite:///complaint.db
import time
%sql drop index if exists state_product_index;
%sql drop index if exists state_index;
%sql analyze

 * sqlite:///complaint.db
Done.
 * sqlite:///complaint.db
Done.


[]

In [4]:
%sql select count(*) from complaints;

 * sqlite:///complaint.db
Done.


count(*)
79468


In [5]:
%sql select * from complaints limit 5;

 * sqlite:///complaint.db
Done.


Date_received,Product,Subproduct,Issue,Subissue,Consumer_narrative,Company_public_response,Company,State,ZIP_code,Submitted_via,Date_sent_to_company,Company_response,Timely_response,Consumer_disputed,Complaint_ID
12/30/2015,Mortgage,Other mortgage,"Loan servicing, payments, escrow account",,,Company chooses not to provide a public response,U.S. Bancorp,TN,38138,,N/A,Referral,01/05/2016,Closed with explanation,Yes
12/12/2015,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",,,Company chooses not to provide a public response,Citibank,NY,13021,,N/A,Referral,12/23/2015,Closed with explanation,Yes
12/02/2015,Mortgage,Other mortgage,"Loan modification,collection,foreclosure",,,,Nationstar Mortgage,MI,49102,,N/A,Referral,12/17/2015,Closed with explanation,Yes
12/02/2015,Bank account or service,Other bank product/service,"Account opening, closing, or management",,,Company chooses not to provide a public response,Wells Fargo & Company,,,,N/A,Referral,12/07/2015,Closed with explanation,Yes
12/22/2015,Mortgage,Conventional fixed mortgage,"Loan servicing, payments, escrow account",,,,Nationstar Mortgage,FL,33484,Older American,Consent not provided,Web,01/06/2016,Closed with explanation,Yes


### Task 1: Query without an index

First, let's start off by writing a query to find the **counts of the top 5 Product, State pairs** in the complaints database (return the product and state as well as the count).  Use the single-line syntax for simple timing so we can see how long the query takes:

In [6]:
%time %sql SELECT product, state, count(*) AS c FROM complaints GROUP BY product, state ORDER BY c DESC LIMIT 5;

 * sqlite:///complaint.db
Done.
CPU times: user 149 ms, sys: 130 ms, total: 278 ms
Wall time: 450 ms


Product,State,c
None,None,13451
Mortgage,CA,3891
Mortgage,FL,2343
Debt collection,None,1654
Mortgage,None,1427


### Task 2: Single search key index

Now create a _single-key_ index such that the above query is faster!  The syntax to create an index in SQL is:
> DROP INDEX IF EXISTS index_name;
> CREATE INDEX index_name ON table(attributes);

In [11]:
%%sql
DROP INDEX IF EXISTS state_index;
CREATE INDEX state_index ON complaints(state);

 * sqlite:///complaint.db
Done.
Done.


[]

In [12]:
%time %sql SELECT product, state, count(*) AS c FROM complaints GROUP BY product, state ORDER BY c DESC LIMIT 5;

 * sqlite:///complaint.db
Done.
CPU times: user 118 ms, sys: 39.5 ms, total: 157 ms
Wall time: 194 ms


Product,State,c
None,None,13451
Mortgage,CA,3891
Mortgage,FL,2343
Debt collection,None,1654
Mortgage,None,1427


### Task 3 (a):

Now, create a _covering_ index for the query and then see how long it takes to run!

In [13]:
%%sql
DROP INDEX IF EXISTS state_product_index;
CREATE INDEX state_product_index ON complaints(state, product);

 * sqlite:///complaint.db
Done.
Done.


[]

In [14]:
%time %sql SELECT product, state, count(*) AS c FROM complaints GROUP BY product, state ORDER BY c DESC LIMIT 5;

 * sqlite:///complaint.db
Done.
CPU times: user 20.1 ms, sys: 4.42 ms, total: 24.5 ms
Wall time: 23.9 ms


Product,State,c
None,None,13451
Mortgage,CA,3891
Mortgage,FL,2343
Debt collection,None,1654
Mortgage,None,1427


### Task 3 (b):

Use EXPLAIN to see if sqlite used/recognized your covering index.  EXPLAIN is an operator that tells SQL to explain its query plan... we'll look into this in more depth later.  For now, the syntax is:
> EXPLAIN QUERY PLAN your_query_here;

In [15]:
%%sql
EXPLAIN QUERY PLAN
SELECT product, state, count(*) AS c
FROM complaints
GROUP BY product, state ORDER BY c DESC LIMIT 5;

 * sqlite:///complaint.db
Done.


id,parent,notused,detail
8,0,0,SCAN complaints USING COVERING INDEX state_product_index
44,0,0,USE TEMP B-TREE FOR ORDER BY


In [18]:
%sql PRAGMA index_list('complaints');

 * sqlite:///complaint.db
Done.


seq,name,unique,origin,partial
0,state_product_index,0,c,0
1,state_index,0,c,0
